Riduzione del dataset

In [ ]:
!pip install tensorflow
!pip install numpy

In [ ]:
import tensorflow as tf
import numpy as np

In [2]:
train_ds = tf.keras.utils.audio_dataset_from_directory(directory='../original/train/audio')

Found 64721 files belonging to 30 classes.


In [3]:
print('Classi:',train_ds.class_names)

Classi: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [4]:
count = np.zeros(len(train_ds.class_names))
for _, labels in train_ds:
    y, _, c = tf.unique_with_counts(labels)
    count[y.numpy()] += c.numpy()

In [5]:
import os
import random
import shutil

In [8]:
def copy_files(files, src_dir, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for file in files:
        shutil.copy(os.path.join(src_dir, file), os.path.join(dest_dir, file))

def downsample_class(class_dir, class_name, target_dir, ratio):
    files = os.listdir(class_dir)
    num_files_to_keep = int(len(files) * ratio)
    files_to_keep = random.sample(files, num_files_to_keep)
    copy_files(files_to_keep, class_dir, os.path.join(target_dir, class_name))

In [9]:
original_dir = '../original/train/audio'
downsampled_dir = '../reduced_dataset/train/audio'

classes = [d for d in os.listdir(original_dir) if os.path.isdir(os.path.join(original_dir, d))]

for class_name in classes:
    class_dir = os.path.join(original_dir, class_name)
    downsample_class(class_dir, class_name, downsampled_dir, 0.20)

print("Dataset ridotto")

Dataset ridotto
